In [7]:
from catboost import CatBoostRegressor, Pool
import pandas as pd
import numpy as np
from sklearn import metrics
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split

In [8]:
train_data = pd.read_csv('/Users/konstantin/MachineLearning/task4/data/train.csv')
test_data = pd.read_csv('/Users/konstantin/MachineLearning/task4/data/test.csv')
#delete ids
train_data = train_data.drop(columns=["Id"])
val_ids = test_data["Id"]  #remember for submission
test_data = test_data.drop(columns=["Id"])
# delete nans
for feature in train_data.columns:
    percent = train_data[feature].isnull().sum() / train_data.shape[0]
    if percent > 0.7:
        train_data = train_data.drop(columns=feature)
        test_data = test_data.drop(columns=feature)
# fill nans(Special for cats)
train_data = train_data.fillna(train_data.median())
test_data = test_data.fillna(test_data.median())
# get categorical features list
cat_features = []
num_feature = 0
for feature in train_data.columns[:-1]:
    if train_data[feature].dtype == 'object': cat_features.append(num_feature)
    num_feature += 1
# split
X_train, X_test, y_train, y_test = train_test_split(train_data.drop(columns='SalePrice').values,
                                                    np.log1p(train_data['SalePrice'].values), test_size=0.2,
                                                    random_state=98987)
# X_train = SimpleImputer(strategy='most_frequent').fit_transform(X_train)
# X_test = SimpleImputer(strategy='most_frequent').fit_transform(X_test)

In [10]:
cat_model = CatBoostRegressor(cat_features=cat_features, depth = 2, loss_function = 'RMSE', iterations = 100000, task_type = "GPU",  devices = '0:1')
train_pool = Pool(X_train, y_train, cat_features=cat_features)
dev_pool = Pool(X_test, y_test, cat_features=cat_features)
cat_model.fit(train_pool, eval_set = dev_pool, early_stopping_rounds = 10, verbose = 0, plot = False)
print('AbsError train:', metrics.mean_absolute_error(cat_model.predict(X_train), y_train))
print('AbsError test:', metrics.mean_absolute_error(cat_model.predict(X_test), y_test))

cat_model.fit(X_train, y_train)
print('Abs error on train data: %1.2f'%metrics.mean_absolute_error(cat_model.predict(X_train), y_train))
print('Abs error on test data %1.2f:'%metrics.mean_absolute_error(cat_model.predict(X_test), y_test))

Abs error on train data: 0.08
Abs error on test data 0.10:
